
# HAM10000 – Concept Reading & Categorization Notebook

Goal of this notebook:
- Read and categorize all *usable concepts* for future CBM work
- Clearly separate:
  - Concepts (C)
  - Targets / labels (Y)
  - Technical / excluded fields

This notebook is ONLY about concepts (no modeling).



## Dataset structure

Expected directory:

```
data/HAM10000/
 ├── ham10000_metadata_2026-02-09.csv
 └── ISIC-images/
```


In [1]:

import pandas as pd
from pathlib import Path

DATA_DIR = Path("data/HAM10000")
META_FILE = DATA_DIR / "ham10000_metadata_2026-02-09.csv"

df = pd.read_csv(META_FILE)
df.head()


,isic_id,attribution,copyright_license,age_approx,anatom_site_general,anatom_site_special,concomitant_biopsy,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_confirm_type,image_manipulation,image_type,lesion_id,melanocytic,sex
0,ISIC_0024306,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,45.0,NaN,NaN,False,Benign,Benign melanocytic proliferations,Nevus,serial imaging showing no change,NaN,dermoscopic,IL_7252831,True,male
1,ISIC_0024307,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,50.0,lower extremity,NaN,False,Benign,Benign melanocytic proliferations,Nevus,serial imaging showing no change,NaN,dermoscopic,IL_6125741,True,male
2,ISIC_0024308,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,55.0,NaN,NaN,False,Benign,Benign melanocytic proliferations,Nevus,serial imaging showing no change,NaN,dermoscopic,IL_3692653,True,female
3,ISIC_0024309,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,40.0,NaN,NaN,False,Benign,Benign melanocytic proliferations,Nevus,serial imaging showing no change,NaN,dermoscopic,IL_0959663,True,male
4,ISIC_0024310,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC,60.0,anterior torso,NaN,True,Malignant,Malignant melanocytic proliferations (Melanoma),"Melanoma, NOS",histopathology,NaN,dermoscopic,IL_8194852,True,male



## 1. Raw columns


In [ ]:

df.columns.tolist()

['isic_id',
 'attribution',
 'copyright_license',
 'age_approx',
 'anatom_site_general',
 'anatom_site_special',
 'concomitant_biopsy',
 'diagnosis_1',
 'diagnosis_2',
 'diagnosis_3',
 'diagnosis_confirm_type',
 'image_manipulation',
 'image_type',
 'lesion_id',
 'melanocytic',
 'sex']


## 2. Diagnosis hierarchy (labels, not concepts)

This dataset contains a three-level diagnostic hierarchy:

- diagnosis_1 : clinical malignancy status
- diagnosis_2 : pathological family
- diagnosis_3 : specific pathological entity

These are the targets (Y), not concepts (C).


In [ ]:

df["diagnosis_1"].value_counts()

diagnosis_1
Benign           9415
Malignant        2156
Indeterminate     149
Name: count, dtype: int64

In [ ]:

df["diagnosis_2"].value_counts()

diagnosis_2
Benign melanocytic proliferations                           7737
Benign epidermal proliferations                             1338
Malignant melanocytic proliferations (Melanoma)             1305
Malignant adnexal epithelial proliferations - Follicular     622
Malignant epidermal proliferations                           229
Benign soft tissue proliferations - Vascular                 180
Benign soft tissue proliferations - Fibro-histiocytic        160
Indeterminate epidermal proliferations                       149
Name: count, dtype: int64

In [ ]:

df["diagnosis_3"].value_counts()

diagnosis_3
Nevus                           7737
Pigmented benign keratosis      1338
Melanoma, NOS                   1305
Basal cell carcinoma             622
Squamous cell carcinoma, NOS     229
Dermatofibroma                   160
Solar or actinic keratosis       149
Name: count, dtype: int64


### Diagnostic meaning

diagnosis_1  → coarse clinical outcome  
diagnosis_2  → pathological group  
diagnosis_3  → fine-grained pathology  

We will later choose which level to use as Y.



## 3. Definition of CBM concepts

We define concepts as *interpretable variables available before diagnosis*.

In this dataset, only metadata-based concepts are available.



### 3.1 Candidate concept columns


In [6]:

concept_columns = [
    "age_approx",
    "sex",
    "anatom_site_general",
    "anatom_site_special",
    "melanocytic",
    "concomitant_biopsy"
]

df[concept_columns].head()


,age_approx,sex,anatom_site_general,anatom_site_special,melanocytic,concomitant_biopsy
0,45.0,male,NaN,NaN,True,False
1,50.0,male,lower extremity,NaN,True,False
2,55.0,female,NaN,NaN,True,False
3,40.0,male,NaN,NaN,True,False
4,60.0,male,anterior torso,NaN,True,True



## 4. Concept categorization

We divide concepts into clinically meaningful groups.



### 4.1 Patient context concepts
- age_approx
- sex


In [7]:

df[["age_approx","sex"]].describe(include="all")


,age_approx,sex
count,11337.000000,11377
unique,NaN,2
top,NaN,male
freq,NaN,6179
mean,52.037135,NaN
std,16.704833,NaN
min,5.000000,NaN
25%,40.000000,NaN
50%,50.000000,NaN
75%,65.000000,NaN



### 4.2 Lesion anatomical context concepts
- anatom_site_general
- anatom_site_special


In [8]:

df["anatom_site_general"].value_counts()


anatom_site_general
lower extremity    2731
posterior torso    2504
anterior torso     1624
upper extremity    1382
head/neck          1255
oral/genital         55
palms/soles           7
Name: count, dtype: int64

In [9]:

df["anatom_site_special"].value_counts().head(15)


anatom_site_special
acral NOS               475
oral or genital          55
acral palms or soles      7
Name: count, dtype: int64


### 4.3 Biological / pathological prior concepts
- melanocytic


In [10]:

df["melanocytic"].value_counts()


melanocytic
True     9042
False    2678
Name: count, dtype: int64


### 4.4 Clinical procedure related concept
- concomitant_biopsy


In [11]:

df["concomitant_biopsy"].value_counts()


concomitant_biopsy
True     6227
False    5493
Name: count, dtype: int64


## 5. Strong relationship warning (concept leakage analysis)

We check how strong the melanocytic concept is with respect to diagnosis.


In [12]:

pd.crosstab(df["melanocytic"], df["diagnosis_3"])


diagnosis_3,Basal cell carcinoma,Dermatofibroma,"Melanoma, NOS",Nevus,Pigmented benign keratosis,Solar or actinic keratosis,"Squamous cell carcinoma, NOS"
melanocytic,,,,,,,
False,622,160,0,0,1338,149,229
True,0,0,1305,7737,0,0,0



## 6. Final concept taxonomy for CBM

C_meta (usable now):

- C_age (from age_approx, later discretized)
- C_sex
- C_anatom_site_general
- C_melanocytic
- C_concomitant_biopsy

Optional / advanced:

- C_anatom_site_special (only if grouped)

Excluded from concepts:

- isic_id
- lesion_id
- attribution
- copyright_license
- image_type
- image_manipulation
- diagnosis_confirm_type



## 7. Label (Y) hierarchy summary

Y can be defined at three levels:

Level 1 (coarse):
- diagnosis_1  → {Benign, Malignant, Indeterminate}

Level 2 (intermediate):
- diagnosis_2  → pathological families

Level 3 (fine-grained):
- diagnosis_3  → concrete pathology classes

This hierarchy enables multi-level CBM experiments.



## 8. CBM formulation for this dataset

X : dermoscopic image

C : metadata concepts
    {age, sex, anatomical site, melanocytic, concomitant biopsy}

Y : diagnosis (chosen level among diagnosis_1, diagnosis_2 or diagnosis_3)

X → C → Y



## 9. Other datasets HAM10000 diagnostic

The kaggle dataset HAM10000 dataset contains 7 diagnostic categories:

| Code | Meaning |
|------|-------|
| nv | Melanocytic nevi |
| mel | Melanoma |
| bkl | Benign keratosis-like lesions |
| bcc | Basal cell carcinoma |
| akiec | Actinic keratoses / intraepithelial carcinoma |
| df | Dermatofibroma |
| vasc | Vascular lesions |

Here is the Link : https://www.kaggle.com/datasets/surajghuwalewala/ham1000-segmentation-and-classification/data

These correspond to the *Target label* Y in CBM terminology.



## 10. Summary

This notebook isolates and categorizes all interpretable concepts available
in the HAM10000 metadata and clearly separates them from diagnostic labels.

This is the conceptual preparation step before building CBM datasets.
